In [ ]:
yolo detect train \
    data=data.yaml \
    model=yolov8n.pt \
    epochs=100 \
    imgsz=640 \
    batch=8 \
    workers=2 \
    device=0 \
    cache=ram \
    exist_ok=True

## box_loss: 0.945 

In [ ]:
nohup yolo detect train \
    data=data.yaml \
    model=yolov8s.pt \
    epochs=100 \
    imgsz=0 \
    batch=8 \              
    workers=2 \            
    device=0 \
    lr0=0.01 \
    patience=10 \
    cache=False \          
    augment=True \
    verbose=True > train_repaired.log 2>&1 &

## box_loss: 0.8774


In [ ]:
nohup yolo detect train \
    resume=True \
    model=runs/detect/train4/weights/last.pt \
    data=data.yaml \
    epochs=100 \
    imgsz=640 \
    batch=4 \             
    workers=1 \            
    device=0 \
    lr0=0.01 \
    patience=10 \
    cache=False \          
    augment=False \        
    val=False \           
    verbose=True > train_light.log 2>&1 &

In [ ]:
yolo train \
  model=yolov11l.pt \
  data=data.yaml \  
  epochs=100 \
  imgsz=480 \
  batch=16 \
  workers=2 \            
  device=0 \
  lr0=0.005 \
  patience=10 \
  cache=False \          
  augment=False \        
  val=False \ 
  project=yolov11_subset_results

In [ ]:
yolo val model=yolov11_subset_results/lr001/weights/best.pt data=data.yaml conf=0.1 iou=0.5 plots=True workers=0 cache=False

In [ ]:
yolo train model=yolo11l data=data.yaml epochs=100 imgsz=640 batch=8 workers=0 device=0 lr0=0.005 augment=True optimizer="AdamW" cls=0.5 pretrained=True  

In [ ]:
import torch
import time 
import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
from torch.utils.data import Dataset, DataLoader
import yaml
from PIL import Image
import os

# Config
data_yaml = 'data.yaml'  # Your dataset config
batch_size = 2
image_size = 512
epochs = 20

# Dataset
class YOLODataset(Dataset):
    def __init__(self, yaml_path, mode='train'):
        with open(yaml_path) as f:
            data = yaml.safe_load(f)
        
        self.img_dir = os.path.join(data['path'], data[mode])
        self.label_dir = self.img_dir.replace('images', 'labels')
        self.images = [f for f in os.listdir(self.img_dir) 
                      if f.endswith(('.jpg', '.png', '.jpeg'))]
        self.classes = data['names']
        
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        # Load image
        img_path = os.path.join(self.img_dir, self.images[idx])
        img = Image.open(img_path).convert('RGB')
        img = torchvision.transforms.functional.to_tensor(img)
        img = torchvision.transforms.functional.resize(img, [image_size]*2)
        
        # Load labels
        label_path = os.path.join(self.label_dir, 
                                os.path.splitext(self.images[idx])[0] + '.txt')
        boxes, labels = [], []
        
        if os.path.exists(label_path):
            with open(label_path) as f:
                for line in f:
                    class_id, xc, yc, w, h = map(float, line.strip().split())
                    # Convert YOLO to Pascal VOC
                    x1 = (xc - w/2) * image_size
                    y1 = (yc - h/2) * image_size
                    x2 = (xc + w/2) * image_size
                    y2 = (yc + h/2) * image_size
                    boxes.append([x1, y1, x2, y2])
                    labels.append(int(class_id) + 1)  # +1 because background is class 0
        
        target = {
            'boxes': torch.tensor(boxes, dtype=torch.float32),
            'labels': torch.tensor(labels, dtype=torch.int64),
            'image_id': torch.tensor([idx]),
            'area': (torch.tensor(boxes)[:, 3] - torch.tensor(boxes)[:, 1]) * 
                    (torch.tensor(boxes)[:, 2] - torch.tensor(boxes)[:, 0]),
            'iscrowd': torch.zeros(len(labels), dtype=torch.int64)
        }
        
        return img, target

# Model
def create_model(num_classes):
    backbone = torchvision.models.mobilenet_v2(weights='DEFAULT').features
    backbone.out_channels = 1280  # MobilenetV2 feature dimension
    
    anchor_generator = AnchorGenerator(
        sizes=((32, 64, 128, 256),),
        aspect_ratios=((0.5, 1.0, 2.0),)
    )
    
    roi_pooler = torchvision.ops.MultiScaleRoIAlign(
        featmap_names=['0'],
        output_size=7,
        sampling_ratio=2
    )
    
    return FasterRCNN(
        backbone,
        num_classes=num_classes,
        rpn_anchor_generator=anchor_generator,
        box_roi_pool=roi_pooler
    )

# Training
def train():
    # Data
    train_set = YOLODataset(data_yaml, 'train')
    train_loader = DataLoader(
        train_set,
        batch_size=batch_size,
        shuffle=True,
        collate_fn=lambda x: tuple(zip(*x))
    )
    
    # Model
    model = create_model(len(train_set.classes) + 1).to('cuda')
    optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)
    
    print(f"\n🚀 Starting training on {len(train_set)} images")
    print(f"📦 Batch size: {batch_size} | 🔄 Total batches: {len(train_loader)}")
    print(f"🔥 Epochs: {epochs} | 💻 Device: {next(model.parameters()).device}\n")

    # Training loop
    for epoch in range(epochs):
        model.train()
        epoch_loss = 0
        start_time = time.time()
        
        for batch_idx, (images, targets) in enumerate(train_loader):
            batch_start = time.time()

            # Move to GPU
            images = [img.to('cuda') for img in images]
            targets = [{k: v.to('cuda') for k, v in t.items()} for t in targets]
            
            # Forward + backward
            optimizer.zero_grad()
            loss_dict = model(images, targets)
            losses = sum(loss for loss in loss_dict.values())

            losses.backward()
            optimizer.step()

            batch_time = time.time() - batch_start
            epoch_loss += losses.item()
            avg_loss = epoch_loss / (batch_idx + 1)

            print(
                f"\rEpoch {epoch+1}/{epochs} | "
                f"Batch {batch_idx+1}/{len(train_loader)} | "
                f"Loss: {losses.item():.3f} (avg: {avg_loss:.3f}) | "
                f"Time: {batch_time:.2f}s/batch | "
                f"Mem: {torch.cuda.memory_allocated()/1e9:.2f}GB",
                end="", flush=True
            )                    

        epoch_time = time.time() - start_time
        print(f"\n✅ Epoch {epoch+1} complete | "
              f"Avg loss: {epoch_loss/len(train_loader):.4f} | "
              f"Time: {epoch_time:.1f}s | "
              f"LR: {optimizer.param_groups[0]['lr']:.2e}\n")

if __name__ == '__main__':
    train()